# Actin & calcium channel segmentation

In [1]:
from tifffile import imread, imwrite
from pathlib import Path
from skimage.transform import resize
import numpy as np
import napari
from tqdm.notebook import tqdm


In [2]:
image_path = Path("../DATA/series003_cCAR_tumor.tif")

image = imread(image_path)
image_actin_channel = image[:, 1, ...]
image_calcium_channel = image[:, 2, ...]
image = image[:, 0, ...]
image.shape

(162, 1412, 1412)

In [3]:
image_resized =  resize(image, (image.shape[0], image.shape[1] // 2, image.shape[2] // 2), anti_aliasing=True)
image_actin_resized = resize(image_actin_channel, (image_actin_channel.shape[0], image_actin_channel.shape[1] // 2, image_actin_channel.shape[2] // 2), anti_aliasing=True)
image_calcium_resized = resize(image_calcium_channel, (image_calcium_channel.shape[0], image_calcium_channel.shape[1] // 2, image_calcium_channel.shape[2] // 2), anti_aliasing=True)

In [4]:
v = napari.Viewer()

In [5]:
v.add_image(image_resized, name="image", colormap="viridis")
v.add_image(image_actin_resized, name="actin", colormap="magma")
v.add_image(image_calcium_resized, name="calcium", colormap="red")

<Image layer 'calcium' at 0x1d76e1d4400>

In [6]:
masks_path = Path("../DATA/tracked")

# Stack all tiff files in the directory into a single 3D numpy array
masks = []
for mask_path in tqdm(masks_path.glob("*.tif")):
    mask = imread(mask_path)
    masks.append(mask)
masks = np.array(masks)

0it [00:00, ?it/s]

In [7]:
masks.shape

(162, 706, 706)

In [8]:
v.add_labels(masks, name="masks")

<Labels layer 'masks' at 0x1d84ebd01f0>

In [ ]:
from pyclesperanto_prototype import difference_of_gaussian
from skimage.segmentation import watershed
from skimage.measure import regionprops

In [10]:
actin_dog = difference_of_gaussian(image_actin_resized, 
                                   sigma1_x=5, sigma2_x=10,
                                   sigma1_y=5, sigma2_y=10,
                                   sigma1_z=0, sigma2_z=0
                                   ) * 1000
actin_dog = np.array(actin_dog, dtype=np.float32)

In [13]:
actin_dog_filtered = np.where(actin_dog < 0.1, 0, actin_dog)
# actin_dog_filtered = np.where(actin_dog_filtered > 0.3, 0, actin_dog_filtered)


In [20]:
actin_dog_filtered = np.array(actin_dog_filtered)


In [ ]:
v.add_image(actin_dog_filtered, name="actin_dog_filtered", colormap="turbo", blending="additive")

In [56]:
# actin_dog_filtered_labels = voronoi_otsu_labeling(actin_dog_filtered, spot_sigma=5, outline_sigma=2)
actin_dog_filtered_labels = np.zeros_like(actin_dog_filtered)
for t, frame in tqdm(enumerate(actin_dog_filtered), total=actin_dog_filtered.shape[0]):
    frame_flipped = np.max(frame) - frame
    frame_flipped = np.where(frame == 0, np.inf, frame_flipped)
    wshed = watershed(frame_flipped)
    actin_dog_filtered_labels[t] = np.where(frame != 0, wshed, 0)

  0%|          | 0/162 [00:00<?, ?it/s]

In [57]:
v.add_labels(actin_dog_filtered_labels.astype(np.uint16), name="actin_filtered_labels")

<Labels layer 'actin_filtered_labels' at 0x1d89bdc5220>

In [58]:
calcium_mask = np.where(image_calcium_resized > 0.003, 1, 0)
calcium_mask = np.array(calcium_mask, dtype=np.uint8)

In [59]:
v.add_labels(calcium_mask)

<Labels layer 'calcium_mask [1]' at 0x1d84253dac0>

In [ ]:
def filter_labels_by_area(masks, min_area=None, max_area=None):
    """
    Filter labels by area using regionprops.
    """
    filtered_labels = np.zeros_like(masks, dtype=np.uint16)
    for t in tqdm(range(masks.shape[0])):
        labels = masks[t]
        props = regionprops(labels)
        for prop in props:
            area = prop.area
            if (min_area is None or area >= min_area) and (max_area is None or area <= max_area):
                filtered_labels[t][labels == prop.label] = prop.label
    return filtered_labels

In [64]:
actin_dog_filtered_labels = filter_labels_by_area(actin_dog_filtered_labels.astype(np.uint16), min_area=100, max_area=None)

In [65]:
v.add_labels(actin_dog_filtered_labels)

<Labels layer 'actin_dog_filtered_labels' at 0x1d840c8b730>